In [ ]:
%load_ext autoreload
%autoreload 2

# Generate subword tokenizer

Use the training data from notebook 100 or preferred tree names to train a subword tokenizer.

This notebook generates 4 different tokenizers. We ended up using just the base tokenizer - the one saved in tokenizer_path.

In [ ]:
from collections import Counter
import json
import random
from typing import List

import boto3
import Levenshtein
import pandas as pd
from tokenizers import models, Tokenizer, trainers, NormalizedString, PreTokenizedString
from tokenizers.normalizers import BertNormalizer
from tokenizers.pre_tokenizers import PreTokenizer, Whitespace
from transformers import PreTrainedTokenizerFast
from tqdm.auto import tqdm

from src.data.filesystem import fopen

In [ ]:
given_surname = 'given'

# run with 500, 1000, 1500, 1800, 2000
vocab_size = 2000
vocab_type = 'f'

triplets_path=f"../data/processed/tree-hr-{given_surname}-triplets-v2-1000.csv.gz"
pref_path = f"s3://familysearch-names/processed/tree-preferred-{given_surname}-aggr.csv.gz"
train_path = f"s3://familysearch-names/processed/tree-hr-{given_surname}-train-v2.csv.gz"
nama_bucket = 'nama-data'

tokenizer_path=f"../data/models/fs-{given_surname}-subword-tokenizer-{vocab_size}{vocab_type}.json"
edit_tokenizer_path=f"../data/models/fs-{given_surname}-edit-subword-tokenizer-{vocab_size}.json"
tokenizer_bigrams_vocab_path = f"../data/models/fs-{given_surname}-tokenizer_vocab_bigrams-{vocab_size}{vocab_type}.json"
edit_tokenizer_bigrams_vocab_path = f"../data/models/fs-{given_surname}-edit_tokenizer_vocab_bigrams-{vocab_size}.json"

## Load data

In [ ]:
# read triplets
triplets_df = pd.read_csv(triplets_path, na_filter=False)
print(len(triplets_df))
triplets_df.head(3)

In [ ]:
train_names_df = pd.read_csv(train_path, na_filter=False)
print(train_names_df.shape)
train_names_df.head(3)

In [ ]:
pref_df = pd.read_csv(pref_path, na_filter=False)
print(len(pref_df))
print(pref_df['frequency'].sum())
pref_df.head(3)

In [ ]:
if vocab_type == 'p':  # use preferred names
    all_names = []
    for row in tqdm(pref_df.to_dict('records')):
        for _ in range(row['frequency']):
            all_names.append(row['name'])
    random.shuffle(all_names)
elif vocab_type == 'f':  # use train names with frequency
    all_names = []
    for row in tqdm(train_names_df.to_dict('records')):
        for _ in range(row['frequency']):
            all_names.append(row['tree_name'])
            all_names.append(row['record_name'])
    random.shuffle(all_names)    
else:  # use train names as a set
    all_names = list(set(train_names_df['tree_name']) | set(train_names_df['record_name']))
print(len(all_names))

## Calculate edit pieces based on anchor-positive pairs

In [ ]:
def generate_edit_pieces(src, tar):
    src_pieces = []
    tar_pieces = []
    opcodes = Levenshtein.opcodes(src, tar)
    for (opcode, src_start, src_end, tar_start, tar_end) in opcodes:
        if opcode == 'equal':
            src_pieces.append(src[src_start:src_end])
            tar_pieces.append(tar[tar_start:tar_end])
        elif opcode == 'delete':
            src_pieces.append(src[src_start:src_end])
        elif opcode == 'insert':
            tar_pieces.append(tar[tar_start:tar_end])
        elif opcode == 'replace':
            src_pieces.append(src[src_start:src_end])
            tar_pieces.append(tar[tar_start:tar_end])
        else:
            print('Unexpected opcode', opcode)
    return ','.join(src_pieces), ','.join(tar_pieces)

In [ ]:
anchor_pos_df = triplets_df[['anchor', 'positive']].drop_duplicates()
len(anchor_pos_df)

In [ ]:
edit_names = set()
for src, tar in tqdm(zip(anchor_pos_df['anchor'], anchor_pos_df['positive'])):
    src, tar = generate_edit_pieces(src, tar)
    edit_names.add(src)
    edit_names.add(tar)
len(edit_names)

## Pre-tokenize by splitting on edit pieces

In [ ]:
class EditPiecePreTokenizer:
    def split(self, i: int, normalized_string: NormalizedString) -> List[NormalizedString]:
        # we need to call `str(normalized_string)` because split expects a str,
        # not a NormalizedString
        return [NormalizedString(s) for s in str(normalized_string).split(',')]
    
    def pre_tokenize(self, pretok: PreTokenizedString):
        return pretok.split(self.split)

## Generate Subwords

In [ ]:
special_tokens = ["[UNK]", "[PAD]", "[CLS]", "[SEP]", "[MASK]"]

### from edit pieces

In [ ]:
edit_tokenizer = Tokenizer(models.WordPiece(unk_token="[UNK]"))
edit_tokenizer.pre_tokenizer = PreTokenizer.custom(EditPiecePreTokenizer())

trainer = trainers.WordPieceTrainer(vocab_size=vocab_size, special_tokens=special_tokens)

In [ ]:
# train tokenizer from edit pieces
def get_edit_names():
    for name in edit_names:
        yield name
        
edit_tokenizer.train_from_iterator(get_edit_names(), trainer=trainer)

In [ ]:
edit_tokenizer.get_vocab()

In [ ]:
# now that the tokenizer has been trained, we don't need the pre-tokenizer any more
# so default it to whitespace
edit_tokenizer.pre_tokenizer = Whitespace()

## Generate subwords from all names

In [ ]:
tokenizer = Tokenizer(models.WordPiece(unk_token="[UNK]"))

trainer = trainers.WordPieceTrainer(vocab_size=vocab_size, special_tokens=special_tokens)

In [ ]:
len(all_names)

In [ ]:
# train tokenizer from all names
def get_all_names_batch(batch_size=10000):
    for i in range(0, len(all_names), batch_size):
        yield all_names[i : i + batch_size]
        
tokenizer.train_from_iterator(get_all_names_batch(), 
                              trainer=trainer, 
                              length=len(all_names))

In [ ]:
tokenizer.get_vocab()

In [ ]:
len(tokenizer.get_vocab())

## Review sample

In [ ]:
sample_size = 100
sample_df = triplets_df.sample(sample_size)

In [ ]:
for anchor, positive in zip(sample_df['anchor'], sample_df['positive']):
    print(anchor, positive)
    print('edit', edit_tokenizer.encode(anchor).tokens, edit_tokenizer.encode(positive).tokens)
    print(' all', tokenizer.encode(anchor).tokens, tokenizer.encode(positive).tokens)

## Save tokenizers

In [ ]:
tokenizer.save(tokenizer_path)  

In [ ]:
edit_tokenizer.save(edit_tokenizer_path)  

## Calculate subwords, subword-bigrams, and lengths

In [ ]:
subword_counter = Counter()
subword_bigrams_counter = Counter()
subword_lengths = Counter()
edit_subword_counter = Counter()
edit_subword_bigrams_counter = Counter()
edit_subword_lengths = Counter()
tokenizer_cache = {}
edit_tokenizer_cache = {}

sample_size = 1000_000

for name in tqdm(all_names[:sample_size]):
    if name not in tokenizer_cache:
        subwords = tokenizer.encode(name).tokens
        tokenizer_cache[name] = subwords
    else:
        subwords = tokenizer_cache[name]
        
    for subword in subwords:
        subword_counter[subword] += 1
    context_subword = 'START'
    subwords.append('END')
    for subword in subwords:
        subword_bigrams_counter[f"{context_subword},{subword}"] += 1
        context_subword = subword
    subword_lengths[len(subwords)] += 1
    
    if name not in edit_tokenizer_cache:
        subwords = edit_tokenizer.encode(name).tokens
        edit_tokenizer_cache[name] = subwords
    else:
        subwords = edit_tokenizer_cache[name]
        
    for subword in subwords:
        edit_subword_counter[subword] += 1
    context_subword = 'START'
    subwords.append('END')
    for subword in subwords:
        edit_subword_bigrams_counter[f"{context_subword},{subword}"] += 1
        context_subword = subword
    edit_subword_lengths[len(subwords)] += 1
    

In [ ]:
subword_counter.most_common()

In [ ]:
edit_subword_counter.most_common()

In [ ]:
print(len(subword_bigrams_counter))
subword_bigrams_counter.most_common(vocab_size)

In [ ]:
print(len(edit_subword_bigrams_counter))
edit_subword_bigrams_counter.most_common(vocab_size)

In [ ]:
edit_subword_lengths

In [ ]:
subword_lengths

## Save bigram tokenizers

In [ ]:
# save tokenizer bigrams vocabulary
tokenizer_bigrams_vocab = {}
ix = 0
for subword in tokenizer.get_vocab():
    tokenizer_bigrams_vocab[subword] = ix
    ix += 1
for bigram, _ in subword_bigrams_counter.most_common(vocab_size):
    tokenizer_bigrams_vocab[bigram] = ix
    ix += 1

print(len(tokenizer_bigrams_vocab))

with fopen(tokenizer_bigrams_vocab_path, 'w') as f:
    json.dump(tokenizer_bigrams_vocab, f)


In [ ]:
# save edit tokenizer bigrams vocabulary
edit_tokenizer_bigrams_vocab = {}
ix = 0
for subword in edit_tokenizer.get_vocab():
    edit_tokenizer_bigrams_vocab[subword] = ix
    ix += 1
for bigram, _ in edit_subword_bigrams_counter.most_common(vocab_size):
    edit_tokenizer_bigrams_vocab[bigram] = ix
    ix += 1
    
print(len(edit_tokenizer_bigrams_vocab))
    
with fopen(edit_tokenizer_bigrams_vocab_path, 'w') as f:
    json.dump(edit_tokenizer_bigrams_vocab, f)


## Test load tokenizer

In [ ]:
tokenizer_path

In [ ]:
loaded_tokenizer = PreTrainedTokenizerFast(tokenizer_file=tokenizer_path)

In [ ]:
loaded_tokenizer.convert_ids_to_tokens(loaded_tokenizer.encode('zacharias'))

In [ ]:
alt_tokenizer_path = '../data/models/fs-given-subword-tokenizer-2000f.json'

In [ ]:
alt_loaded_tokenizer = PreTrainedTokenizerFast(tokenizer_file=alt_tokenizer_path)

In [ ]:
alt_loaded_tokenizer.convert_ids_to_tokens(alt_loaded_tokenizer.encode('zacharias'))

In [ ]:
sample_size = 20
sample_df = triplets_df.sample(sample_size)

for anchor in sample_df['anchor']:
    tokens = loaded_tokenizer.convert_ids_to_tokens(loaded_tokenizer.encode(anchor))
    alt_tokens = alt_loaded_tokenizer.convert_ids_to_tokens(alt_loaded_tokenizer.encode(anchor))
    if tokens != alt_tokens:
        print(anchor, tokens, alt_tokens)

In [ ]:
loaded_tokenizer.convert_ids_to_tokens(loaded_tokenizer.encode('jewel'))

In [ ]:
alt_loaded_tokenizer.convert_ids_to_tokens(alt_loaded_tokenizer.encode('jewel'))